## 2023 06/16 REMBG

*Last Updated*: 2023-06-16

### Authors
* Hannah Zhang (hannahzhang@ucsb.edu)


### Overview
This Jupyter notebook is intended to demonstrate

* generating hand outline from image
* green screen removal

### Key Results

The key results of this experiment are ...

- Rembg algorithm is the best method for green screen removal, generating quick, accurate results for foreground extraction with a simple call function

In [ ]:
# Import libraries

import os
import cv2
import glob
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
#load sample images
images = []
def load_images(foldername):
    for filename in os.listdir(foldername):
        if filename == ".DS_Store":
            continue
        input = Image.open(os.path.join(foldername,filename))
        print(filename)
        
        if input is not None:
            images.append(input)
    return images
img_list = load_images("/Users/hannahzhang/Downloads/test_ims/")

### Rembg Algorithm

In [17]:
from rembg import remove
import easygui
from PIL import Image

In [19]:
removed_ims = []

for im in images:
    output = remove(im)
    removed_ims.append(output)
    #output.show()

### Deeplabv3_resnet50

- model takes a long time to run
- didn't produce desirable results

In [ ]:
def load_model():
  model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
  model.eval()
  return model

In [ ]:
def make_transparent_foreground(pic, mask):
  b, g, r = cv2.split(np.array(pic).astype('uint8'))
  a = np.ones(mask.shape, dtype='uint8') * 255
  alpha_im = cv2.merge([b, g, r, a], 4)
  bg = np.zeros(alpha_im.shape)
  new_mask = np.stack([mask, mask, mask, mask], axis=2)
  foreground = np.where(new_mask, alpha_im, bg).astype(np.uint8)

  return foreground

def remove_background(model, input_image):
  input_image = Image.open(input_image)
  preprocess = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0) 

  if torch.cuda.is_available():
      input_batch = input_batch.to('cuda')
      model.to('cuda')

  with torch.no_grad():
      output = model(input_batch)['out'][0]
  output_predictions = output.argmax(0)
  
  mask = output_predictions.byte().cpu().numpy()
  background = np.zeros(mask.shape)
  bin_mask = np.where(mask, 255, background).astype(np.uint8)

  foreground = make_transparent_foreground(input_image ,bin_mask)

  return foreground, bin_mask

In [ ]:
deeplab_model = load_model()


In [ ]:
foreground, bin_mask = remove_background(deeplab_model, input_image)

Erosion and Dilation

In [ ]:
img = cv2.imread(input_image)
  
kernel = np.ones((5, 5), np.uint8)
  
img_erosion = cv2.erode(img, kernel, iterations=1)
img_dilation = cv2.dilate(img, kernel, iterations=1)
  
cv2.imshow('Input', img)
cv2.imshow('Erosion', img_erosion)
cv2.imshow('Dilation', img_dilation)
  
cv2.waitKey(0)


In [ ]:

def process(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_canny = cv2.Canny(img_gray, 12, 54)
    kernel = np.ones((3, 3))
    img_dilate = cv2.dilate(img_canny, kernel, iterations=10)
    img_erode = cv2.erode(img_dilate, kernel, iterations=8)
    return img_erode

img = cv2.imread(input_image)
contours, _ = cv2.findContours(process(img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
cv2.imshow("Image", img)
cv2.waitKey(0)